In [2]:
import pandas as pd
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Paths to directories
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(results_dir, exist_ok=True)

# Load Working Tables for every optimization try
WorkingTables_Path = 'C:/Users/Nik/Documents/GitHub/Thesis/CSV/Working Tables'
DF_m2_Residential = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Residential.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Industrial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Industrial.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Commercial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Commercial.csv'), sep=",", on_bad_lines='skip', header=0)

Residential_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Residential_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Industrial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Industrial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Commercial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Commercial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Load New Consumption Curves
Residential_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Residential_Consumption_Optimized_From_Switched.csv', index_col=0, parse_dates=True)
Industrial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Industrial_Consumption_Optimized_From_Switched.csv', index_col=0, parse_dates=True)
Commercial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Commercial_Consumption_Optimized_From_Switched.csv', index_col=0, parse_dates=True)

DF_Production = pd.read_csv(os.path.join(Data_path_Production, '2019_Production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Use 'full_id' as the primary identifier
residential_users = DF_m2_Residential['full_id'].astype(str).tolist()
industrial_users = DF_m2_Industrial['full_id'].astype(str).tolist()
commercial_users = DF_m2_Commercial['full_id'].astype(str).tolist()

# Define the yearlist based on the number of hours
yearlist = DF_Production.index

# Fixed consumption area to achieve
Fixed_Res_Consumption_Area = 500000  # Example value
Fixed_Ind_Consumption_Area = 10000   # Example value
Fixed_Com_Consumption_Area = 20000   # Example value

# Create a new Gurobi model
m = gp.Model("Optimization_Model_ConsumptionProduction")

# Production and Consumption Binary Variables [0;1]
binary_vars_production_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_production_residential")
binary_vars_consumption_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_consumption_residential")

binary_vars_production_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_production_industrial")
binary_vars_consumption_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_consumption_industrial")

binary_vars_production_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_production_commercial")
binary_vars_consumption_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_consumption_commercial")

# Auxiliary variable for absolute deviation
deviation = m.addVars(yearlist, lb=0, vtype=GRB.CONTINUOUS, name="deviation")

# Penalty coefficient for not selecting users
penalty_coefficient = 100  # Adjust this value to control the penalization effect

# Calculate the difference between production and consumption for each hour
difference = {}
for t in yearlist:
    try:
        difference[t] = (
            gp.quicksum(binary_vars_production_residential[user] * Residential_user_production_df.loc[t, user] for user in residential_users if user in Residential_user_production_df.columns) -
            gp.quicksum(binary_vars_consumption_residential[user] * Residential_Consumption[f'Residential_User_{user}_Mixed'].loc[t] * DF_m2_Residential.loc[DF_m2_Residential['full_id'] == user, 'Area'].values[0] for user in residential_users if f'Residential_User_{user}_Mixed' in Residential_Consumption.columns) +
            gp.quicksum(binary_vars_production_industrial[user] * Industrial_user_production_df.loc[t, user] for user in industrial_users if user in Industrial_user_production_df.columns) -
            gp.quicksum(binary_vars_consumption_industrial[user] * Industrial_Consumption[f'Industrial_User_{user}_Mixed'].loc[t] * DF_m2_Industrial.loc[DF_m2_Industrial['full_id'] == user, 'Area'].values[0] for user in industrial_users if f'Industrial_User_{user}_Mixed' in Industrial_Consumption.columns) +
            gp.quicksum(binary_vars_production_commercial[user] * Commercial_user_production_df.loc[t, user] for user in commercial_users if user in Commercial_user_production_df.columns) -
            gp.quicksum(binary_vars_consumption_commercial[user] * Commercial_Consumption[f'Commercial_User_{user}_Mixed'].loc[t] * DF_m2_Commercial.loc[DF_m2_Commercial['full_id'] == user, 'Area'].values[0] for user in commercial_users if f'Commercial_User_{user}_Mixed' in Commercial_Consumption.columns)
        )
    except KeyError as e:
        print(f"KeyError at time {t}: {e}")

# Add constraint to define the absolute value of the deviation
for t in yearlist:
    m.addConstr(deviation[t] >= difference[t], name=f"PosDeviation_{t}")
    m.addConstr(deviation[t] >= -difference[t], name=f"NegDeviation_{t}")

# Constraint: Total selected consumption area must equal or exceed the fixed value
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] * DF_m2_Residential.loc[DF_m2_Residential['full_id'] == user, 'Area'].values[0] for user in residential_users) >= Fixed_Res_Consumption_Area,
    name="FixedResConsumptionArea"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_industrial[user] * DF_m2_Industrial.loc[DF_m2_Industrial['full_id'] == user, 'Area'].values[0] for user in industrial_users) >= Fixed_Ind_Consumption_Area,
    name="FixedIndConsumptionArea"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_commercial[user] * DF_m2_Commercial.loc[DF_m2_Commercial['full_id'] == user, 'Area'].values[0] for user in commercial_users) >= Fixed_Com_Consumption_Area,
    name="FixedComConsumptionArea"
)

# Objective function: minimize the deviation and penalize unselected users
penalty = (
    penalty_coefficient * (
        gp.quicksum(1 - binary_vars_production_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_consumption_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_production_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_consumption_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_production_commercial[user] for user in commercial_users) +
        gp.quicksum(1 - binary_vars_consumption_commercial[user] for user in commercial_users)
    )
)

# Minimize deviation and add penalization
m.setObjective(gp.quicksum(deviation[t] for t in yearlist) + penalty, GRB.MINIMIZE)

# Set solver parameters
m.Params.MIPGap = 0.005

# Optimize the model
m.optimize()

# Display the results
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

print(f'Objective: {m.objVal}')

# Extract results
RP_values = {user: binary_vars_production_residential[user].X for user in residential_users}
CP_values = {user: binary_vars_consumption_residential[user].X for user in residential_users}
IP_values = {user: binary_vars_production_industrial[user].X for user in industrial_users}
CIP_values = {user: binary_vars_consumption_industrial[user].X for user in industrial_users}
CP_values_com = {user: binary_vars_production_commercial[user].X for user in commercial_users}
CIP_values_com = {user: binary_vars_consumption_commercial[user].X for user in commercial_users}

# Create new DataFrames with the chosen binary variables
DF_m2_Residential['Chosen_Production'] = [RP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Residential['Chosen_Consumption'] = [CP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Industrial['Chosen_Production'] = [IP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Industrial['Chosen_Consumption'] = [CIP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Commercial['Chosen_Production'] = [CP_values_com[user] for user in DF_m2_Commercial['full_id']]
DF_m2_Commercial['Chosen_Consumption'] = [CIP_values_com[user] for user in DF_m2_Commercial['full_id']]

# Extract chosen users based on optimization results
chosen_residential_production = DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1]
chosen_residential_consumption = DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1]
chosen_industrial_production = DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1]
chosen_industrial_consumption = DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1]
chosen_commercial_production = DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1]
chosen_commercial_consumption = DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1]

# Save the results to CSV files
chosen_residential_production.to_csv(os.path.join(results_dir, 'chosen_residential_production.csv'))
chosen_residential_consumption.to_csv(os.path.join(results_dir, 'chosen_residential_consumption.csv'))
chosen_industrial_production.to_csv(os.path.join(results_dir, 'chosen_industrial_production.csv'))
chosen_industrial_consumption.to_csv(os.path.join(results_dir, 'chosen_industrial_consumption.csv'))
chosen_commercial_production.to_csv(os.path.join(results_dir, 'chosen_commercial_production.csv'))
chosen_commercial_consumption.to_csv(os.path.join(results_dir, 'chosen_commercial_consumption.csv'))

# Calculate the number of chosen users versus the total number of users for each category
num_residential_chosen_production = len(chosen_residential_production)
num_residential_total = len(DF_m2_Residential)

num_residential_chosen_consumption = len(chosen_residential_consumption)
num_industrial_chosen_production = len(chosen_industrial_production)
num_industrial_total = len(DF_m2_Industrial)

num_industrial_chosen_consumption = len(chosen_industrial_consumption)
num_commercial_chosen_production = len(chosen_commercial_production)
num_commercial_total = len(DF_m2_Commercial)

num_commercial_chosen_consumption = len(chosen_commercial_consumption)

# Print the amounts
print(f'Residential Production: Chosen = e{num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')


KeyboardInterrupt: 